In [101]:
import pandas as pd


# pd.set_option('display.max_row', 30000)

In [102]:
df_road = pd.read_csv("/Users/2020/road-to-dong/pos00005m.csv")
df_dong = pd.read_csv("/Users/2020/road-to-dong/pos00006m.csv")


df_road.info()
df_dong.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6318334 entries, 0 to 6318333
Data columns (total 14 columns):
 #   Column       Dtype  
---  ------       -----  
 0   bld_mngm_no  object 
 1   rd_nm        object 
 2   rd_eng_nm    object 
 3   bld_nm       object 
 4   bld_eng_nm   float64
 5   bld_hmno     int64  
 6   bld_hsno     int64  
 7   lgdng_cd     int64  
 8   adstrd_cd    float64
 9   san_yn       int64  
 10  hmno_ltno    int64  
 11  hsno_ltno    int64  
 12  zipcd        float64
 13  rd_mngm_no   object 
dtypes: float64(3), int64(6), object(5)
memory usage: 674.9+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21664 entries, 0 to 21663
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   lgdng_cd      21664 non-null  int64  
 1   ctpv_nm       21664 non-null  object 
 2   ctgg_nm       21664 non-null  object 
 3   adstrd_nm     21664 non-null  object 
 4   adstrd_en_nm  21663 non-null  object

In [103]:
# pd_series 예시 : df_road['rd_nm']
def road_to_dong(pd_series):
    for index in range(len(pd_series)):
        temp=pd_series[index]
        df_temp = df_road[df_road['rd_nm']==temp]
        adstrd_cd = df_temp['adstrd_cd'].iloc[0]
        df_temp = df_dong[df_dong['admn_inst_cd']==adstrd_cd]
        adstrd_nm = df_temp.iloc[0]['adstrd_nm']
        pd_series[index] = adstrd_nm
    return pd_series

In [113]:
df_road_sample = df_road['rd_nm'].copy()[:50]

In [114]:
df_road_sample

0          가운로1길
1          가운로1길
2          가운로1길
3       경춘로859번길
4        덕송2로6번길
5       별내5로23번길
6            인주로
7            대미길
8           신상2길
9           낙동4길
10         다산산단로
11          개경포로
12          쾌빈2길
13          쾌빈2길
14      초전북로52번길
15      사곡로124번길
16      연꽃로165번길
17       삼일로95번길
18           상미로
19          볼음도길
20      시도로104번길
21          청하동길
22    봉수대로1568번길
23          어사대로
24          봉선1로
25      영흥북로35번길
26           사동길
27          서창2길
28         월드컵서로
29         풍암중앙로
30     회재로1293번길
31          서문대로
32          풍서우로
33          신만산길
34      영흥로722번길
35          영흥서로
36           영흥로
37      영흥로544번길
38          천변좌로
39     천변좌로382번길
40          봉주5길
41           회재로
42          천변좌로
43           서동로
44           사직길
45           구성로
46      살곶이길86번길
47      원적로449번길
48           덕목길
49           설성로
Name: rd_nm, dtype: object

In [115]:
len(df_road_sample)

50

In [116]:
df_get_dong = road_to_dong(df_road_sample)

In [117]:
df_get_dong

0         다산2동
1         다산2동
2         다산2동
3          금곡동
4          별내동
5          별내동
6          인주면
7          화동면
8          낙동면
9     낙동면동부출장소
10         다산면
11         성산면
12        대가야읍
13        대가야읍
14         초장동
15         수곡면
16         정촌면
17         정촌면
18         미천면
19    서도면볼음출장소
20         북도면
21         강화읍
22         검단동
23        계산1동
24        봉선1동
25         영흥면
26         서창동
27         서창동
28         풍암동
29         풍암동
30        백운2동
31        주월1동
32         서창동
33         대촌동
34         영흥면
35         영흥면
36         영흥면
37         영흥면
38         사직동
39         사직동
40        주월2동
41        주월1동
42         사직동
43         사직동
44         사직동
45         사직동
46         서신면
47         신둔면
48         우정읍
49         설성면
Name: rd_nm, dtype: object

In [92]:
# pd_series 예시 : df_road['rd_nm']
def road_to_dong(pd_series):
    for index in range(len(pd_series)):
        adstrd_cd = df_road[df_road['rd_nm']==pd_series[index]]['adstrd_cd'].iloc[0]
        pd_series[index] = df_dong[df_dong['admn_inst_cd']==adstrd_cd].iloc[0]['adstrd_nm']
    return pd_series

IndexError: single positional indexer is out-of-bounds

In [119]:
df_road[df_road['rd_nm']=='테헤란로']

,bld_mngm_no,rd_nm,rd_eng_nm,bld_nm,bld_eng_nm,bld_hmno,bld_hsno,lgdng_cd,adstrd_cd,san_yn,hmno_ltno,hsno_ltno,zipcd,rd_mngm_no
510209,1168010100108220000024019,테헤란로,Teheran-ro,명선상가빌딩,NaN,103,0,1168010100,1.168064e+09,0,822,0,135933.0,1168010100108220000024019
524843,1168010600109960000012055,테헤란로,Teheran-ro,슈페리어사옥,NaN,614,0,1168010600,1.168061e+09,0,996,0,135502.0,1168010600109960000012055
761214,1168010100107050029022378,테헤란로,Teheran-ro,NaN,NaN,339,0,1168010100,1.168064e+09,0,705,30,NaN,1168010100107050029022378
1610171,1168010500101440022016954,테헤란로,Teheran-ro,삼영빌딩,NaN,437,0,1168010500,1.168059e+09,0,144,22,135877.0,1168010500101440022016954
1991571,1168010100107360001023747,테헤란로,Teheran-ro,캐피탈타워,NaN,142,0,1168010100,1.168064e+09,0,736,1,135924.0,1168010100107360001023747
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5990265,1168010100108040000000008,테헤란로,Teheran-ro,NaN,NaN,206,2,1168010100,1.168065e+09,0,804,0,NaN,1168010100108040000000008
6171477,1168010100107200001000001,테헤란로,Teheran-ro,NaN,NaN,232,0,1168010100,1.168065e+09,0,720,1,135920.0,1168010100107200001000001
6190502,1168010100106780039000001,테헤란로,Teheran-ro,NaN,NaN,211,0,1168010100,1.168064e+09,0,678,39,135915.0,1168010100106780039000001
6190615,1168010500101410031017036,테헤란로,Teheran-ro,NaN,NaN,407,0,1168010500,1.168059e+09,0,141,31,135876.0,1168010500101410031017036


In [82]:
df_get_dong

0        다산2동
1        다산2동
2        다산2동
3         금곡동
4         별내동
5         별내동
6         인주면
7         화동면
8         낙동면
9    낙동면동부출장소
Name: rd_nm, dtype: object

In [ ]:
# pd_series 예시 : df_road['rd_nm']
def road_to_dong(pd_series):
    for index in pd_series.index:
        adstrd_cd = df_road[df_road['rd_nm']==pd_series[index]]['adstrd_cd'].iloc[0]
        pd_series[index] = df_dong[df_dong['admn_inst_cd']==adstrd_cd].iloc[0]['adstrd_nm']
        return pd_series

In [67]:
# pd_series 예시 : df_road['rd_nm']
def road_to_adstrd_cd(pd_series):
    for index in pd_series.index:
        pd_series[index] = df_road[df_road['rd_nm']==pd_series[index]]['adstrd_cd'].iloc[0]
        return pd_series

In [68]:
df_get_dong = road_to_adstrd_cd(df_road_sample)


<ipython-input-67-c4c6fa345193>:4: FutureWarning: Automatic reindexing on DataFrame vs Series comparisons is deprecated and will raise ValueError in a future version.  Do `left, right = left.align(right, axis=1, copy=False)` before e.g. `left == right`
  pd_series[index] = df_road[df_road['rd_nm']==pd_series[index]]['adstrd_cd'].iloc[0]


KeyboardInterrupt: 

In [22]:
df_road_sample.index

RangeIndex(start=0, stop=300, step=1)

In [25]:
adstrd_cd = df_road[df_road['rd_nm']==df_road_sample[0]]['adstrd_cd'].iloc[0]


In [38]:
adstrd_cd

4136056500.0

In [49]:
df_road_sample[0] = df_dong[df_dong['admn_inst_cd']==adstrd_cd]

In [50]:
df_road_sample

0              lgdng_cd ctpv_nm ctgg_nm adstrd_nm    ...
1                                                  가운로1길
2                                                  가운로1길
3                                               경춘로859번길
4                                                덕송2로6번길
                             ...                        
295                                           일주서로3000번길
296                                           일주서로3000번길
297                                           일주서로3000번길
298                                               도금단지2길
299                                             무섬로234번길
Name: rd_nm, Length: 300, dtype: object

In [64]:
df123 = df_dong[df_dong['admn_inst_cd']==adstrd_cd].iloc[0]['adstrd_nm']

In [65]:
df123

'다산2동'

In [18]:
# df_dong_test = road_to_dong(df_road_sample)

In [19]:
# df_dong_test

0              lgdng_cd ctpv_nm ctgg_nm adstrd_nm    ...
1                                                  가운로1길
2                                                  가운로1길
3                                               경춘로859번길
4                                                덕송2로6번길
                             ...                        
295                                           일주서로3000번길
296                                           일주서로3000번길
297                                           일주서로3000번길
298                                               도금단지2길
299                                             무섬로234번길
Name: rd_nm, Length: 300, dtype: object